In [21]:
import pandas as pd
import json

In [2]:
col = ["indicateur", "period_date", "valeur", "mesure", "short_indic", "dep", "reg"]
df = pd.read_csv("/home/guillaume/Documents/testVue/covid19-dashboard-widgets/donnee_jupiter/pp_dep.csv", usecols=col, sep=";")
df['period_date'] = pd.to_datetime(df['period_date'])

In [3]:
df.columns

Index(['indicateur', 'period_date', 'valeur', 'mesure', 'short_indic', 'dep',
       'reg'],
      dtype='object')

In [4]:
indicList = df.short_indic.unique()
dfList = []
for indic in indicList:
    dfList.append(df.loc[df.short_indic == indic])

In [23]:
# create objects for json
listObject = []

def evolVal(valI, valE):
    if (valE == 0):
        return 0
    elif(valI != valE):
        return valI - valE
    else:
        return 0

def evolPercent(ev, val):
    round((ev / val)*100,2)


for dff in dfList:
    data = {}
    
    data["nom"] = dff["indicateur"].iloc[0]
    data["unite"] = dff["short_indic"].iloc[0]
    
    # FRANCE
    most_recent_date = dff['period_date'].max()
    dfDate = dff.loc[dff.period_date == most_recent_date]
    sumValue = dfDate["valeur"].sum()
    
    #calcul evolution % :
    dfevol = pd.DataFrame()
    dfevol = dff.copy() 
    dfevol.drop(dfevol.loc[dfevol['period_date'] == most_recent_date].index, inplace=True)
    most_recent_date2 = dfevol['period_date'].max()
    dfDatee = dfevol.loc[dfevol.period_date == most_recent_date2]
    sumValue2 = dfDatee["valeur"].sum()
    evol = evolVal(sumValue, sumValue2)
    evol_percent = evolPercent(evol, sumValue2)
    
    #fabrication json france :
    france = {}
    france["last_value"] = sumValue
    france["last_date"] = str(most_recent_date) #.to_pydatetime()
    france["evol"] = evol
    france["evol_percentage"] = evol_percent
    france["evol_color"] = "red"
    france["level"] = "nat"
    france["code_level"] = "fra"
    c = []
    c.append(france)
    data["france"] = c
    
    # REGION
    
    regionList = []
    most_recent_date = ""
    reg = []
    reg = dff.reg.unique()
    for region in reg:
        dfreg = dff.loc[dff.reg == region]
        most_recent_date = dfreg['period_date'].max()
        sumValueReg = dfreg["valeur"].sum()
        
        #calcul evolution % :
        dfevol = pd.DataFrame()
        dfevol = dfreg.copy() 
        dfevol.drop(dfevol.loc[dfevol['period_date'] == most_recent_date].index, inplace=True)
        most_recent_date2 = dfevol['period_date'].max()
        dfDatee = dfevol.loc[dfevol.period_date == most_recent_date2]
        sumValue2 = dfDatee["valeur"].sum()
        evol = 0
        evol_percent = 0
        evol = evolVal(sumValue, sumValue2)
        evol_percent = evolPercent(evol, sumValue2)
        
        #fabrication json region :
        regionDict = {}
        regionDict["last_value"] = sumValueReg
        regionDict["last_date"] = str(most_recent_date) #.to_pydatetime()
        regionDict["evol"] = evol
        regionDict["evol_percentage"] = evol_percent
        regionDict["evol_color"] = "red"
        regionDict["level"] = "reg"
        regionDict["code_level"] = region
        regionList.append(regionDict)
        
    data["regions"] = regionList
    
    # DEPARTEMENTS
    
    departementList = []
    most_recent_date = ""
    dep = []
    dep = dff.dep.unique()
    for dpt in dep:
        dfdep = dff.loc[dff.reg == dpt]
        most_recent_date = dfdep['period_date'].max()
        sumValueDep = dfdep["valeur"].sum()
        
        #calcul evolution % :
        dfevol = pd.DataFrame()
        dfevol = dfdep.copy() 
        dfevol.drop(dfevol.loc[dfevol['period_date'] == most_recent_date].index, inplace=True)
        most_recent_date2 = dfevol['period_date'].max()
        dfDatee = dfevol.loc[dfevol.period_date == most_recent_date2]
        sumValue2 = dfDatee["valeur"].sum()
        evol = 0
        evol_percent = 0
        evol = evolVal(sumValue, sumValue2)
        evol_percent = evolPercent(evol, sumValue2)
        
        #fabrication json departement :
        depDict = {}
        depDict["last_value"] = sumValueDep
        depDict["last_date"] = str(most_recent_date)
        depDict["evol"] = evol
        depDict["evol_percentage"] = evol_percent
        depDict["evol_color"] = "red"
        depDict["level"] = "reg"
        depDict["code_level"] = dpt
        departementList.append(depDict)
        
    data["departements"] = departementList
    
    listObject.append(data)

    

<ipython-input-23-7533bb19b063>:13: RuntimeWarning: invalid value encountered in double_scalars
  round((ev / val)*100,2)


In [24]:

for jsonObj in listObject:
    name = jsonObj["unite"].replace(" ", "_")
    path = "/home/guillaume/Documents/testVue/covid19-dashboard-widgets/donnee_jupiter/json/" + name +".json"
    with open(path, "w", encoding="utf8") as f:
        json.dump(jsonObj, f, ensure_ascii=False)